In [ ]:
pip install tweepy transformers python-dotenv

In [ ]:
import tweepy
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from dotenv import load_dotenv
import os

# Load environment variables from .env file
load_dotenv()

# Step 1: Fetch the latest 10 tweets from a user
def fetch_latest_tweets(username, client, max_results=10):
    try:
        # Resolve the username to user ID
        response = client.get_user(username=username)
        if response.data:
            user_id = response.data.id
            print(f"Resolved User ID: {user_id}")
        else:
            raise Exception("Failed to resolve username. User not found.")

        # Fetch the latest tweets
        tweets = client.get_users_tweets(
            id=user_id,
            max_results=max_results,
            tweet_fields=["created_at", "text"],
        )
        if tweets.data:
            return [tweet.text for tweet in tweets.data]
        else:
            print("No tweets found for this user.")
            return []
    except tweepy.TweepyException as e:
        print(f"Tweepy Error: {e}")
        return []
    except Exception as ex:
        print(f"Error: {ex}")
        return []

# Step 2: Summarize tweets using a pre-trained model
def summarize_tweets(tweets, model_name="t5-small"):
    try:
        # Load the summarization model
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

        # Combine tweets into a single text input
        input_text = "Summarize the following tweets into a single post:\n" + "\n".join(tweets)

        # Tokenize the input text
        inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True)

        # Generate the summary
        summary_ids = model.generate(
            inputs.input_ids,
            max_length=50,           # Maximum length of the summary
            num_beams=5,             # Beam search for better quality
            early_stopping=True      # Stop early when the result is stable
        )

        # Decode and return the summary
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        return summary
    except Exception as ex:
        print(f"Error in summarization: {ex}")
        return "Error: Unable to generate summary."

# Step 3: Post the summary to your Twitter timeline
def post_summary_to_timeline(summary, client):
    try:
        response = client.create_tweet(text=summary)
        print(f"Summary posted successfully: {response.data['id']}")
    except tweepy.TweepyException as e:
        print(f"Error while posting summary: {e}")

# Step 4: Main function to integrate fetching, summarizing, and posting
def main():
    # Load Twitter API credentials from .env
    BEARER_TOKEN = os.getenv("TWITTER_BEARER_TOKEN")
    CONSUMER_KEY = os.getenv("TWITTER_CONSUMER_KEY")
    CONSUMER_SECRET = os.getenv("TWITTER_CONSUMER_SECRET")
    ACCESS_TOKEN = os.getenv("TWITTER_ACCESS_TOKEN")
    ACCESS_TOKEN_SECRET = os.getenv("TWITTER_ACCESS_TOKEN_SECRET")

    if not all([BEARER_TOKEN, CONSUMER_KEY, CONSUMER_SECRET, ACCESS_TOKEN, ACCESS_TOKEN_SECRET]):
        print("Error: Missing Twitter API credentials in .env file.")
        return

    client = tweepy.Client(
        bearer_token=BEARER_TOKEN,
        consumer_key=CONSUMER_KEY,
        consumer_secret=CONSUMER_SECRET,
        access_token=ACCESS_TOKEN,
        access_token_secret=ACCESS_TOKEN_SECRET
    )

    # Username to fetch tweets from
    username = "deepakgudla1357"  # Replace with the desired username

    print(f"Fetching tweets from user: {username}")
    tweets = fetch_latest_tweets(username, client)

    if tweets:
        print("\nFetched Tweets:")
        for tweet in tweets:
            print(f"- {tweet}")

        print("\nGenerating summary...")
        summary = summarize_tweets(tweets)
        print("\nSummary:")
        print(summary)

        print("\nPosting summary to Twitter timeline...")
        post_summary_to_timeline(summary, client)
    else:
        print("No tweets available for summarization.")

if __name__ == "__main__":
    main()
